In [ ]:
!pip install diffusers transformers accelerate

In [ ]:
!pip install torch torchvision

In [ ]:
!pip install pillow

In [ ]:
!sudo apt-get install -y fonts-nanum

In [ ]:
!sudo fc-cache -fv

In [ ]:
!rm ~/.cache/matplotlib -rf

In [ ]:
from diffusers import DiffusionPipeline
import torch
import matplotlib.pyplot as plt
from PIL import Image
import os
import time
import gc
import psutil
import requests
import json

In [ ]:
plt.rc('font', family='NanumBarunGothic')

In [ ]:
# 시스템 정보 확인
print("=== 시스템 정보 ===")
print(f"GPU 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name()}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"RAM: {psutil.virtual_memory().total / 1e9:.1f} GB")

# GPU 런타임 확인
if not torch.cuda.is_available():
    print("⚠️  GPU가 활성화되지 않았습니다!")
    print("Runtime → Change runtime type → Hardware accelerator → GPU 선택")

In [ ]:
# 메모리 정리
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# 코랩에 최적화된 파이프라인 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 더 가벼운 설정으로 로드
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,  # 메모리 절약 (중요!)
    use_safetensors=True,
    variant="fp16",
    low_cpu_mem_usage=True  # CPU 메모리 절약
)

# GPU로 이동
pipe = pipe.to(device)

# LoRA 로드
pipe.load_lora_weights("alvdansen/phantasma-anime")

# 코랩에서 중요한 최적화들
pipe.enable_model_cpu_offload()  # 모델을 필요할 때만 GPU로
pipe.enable_vae_slicing()        # VAE 메모리 절약
pipe.enable_vae_tiling()         # VAE 타일링
pipe.enable_attention_slicing(1) # Attention 슬라이싱

# xFormers 사용 (코랩에 보통 설치되어 있음)
try:
    pipe.enable_xformers_memory_efficient_attention()
    print("✅ xFormers 최적화 적용됨")
except:
    print("⚠️  xFormers 사용 불가")

In [ ]:
CLIENT_ID = ''
CLIENT_SECRET = ''

In [ ]:
# API 키가 설정되지 않은 경우 경고
if CLIENT_ID == "YOUR_CLIENT_ID" or CLIENT_SECRET == "YOUR_CLIENT_SECRET":
    print("⚠️  Papago API 키가 설정되지 않았습니다!")
    print("   네이버 개발자 센터에서 API 키를 발급받아 설정해주세요.")
    print("   현재는 번역 없이 원문이 그대로 prompt에 저장됩니다.")
    USE_PAPAGO = False
else:
    USE_PAPAGO = True

In [ ]:
def papago_translate(text: str, source: str = "ko", target: str = "en", timeout: int = 5) -> str:
    """
    Papago NMT로 번역합니다. (기본: 한국어 -> 영어)
    필요: USE_PAPAGO, CLIENT_ID, CLIENT_SECRET
    """
    if not globals().get("USE_PAPAGO"):
        print("⚠️ API 키 미설정으로 번역을 건너뜁니다.")
        return text

    url = "https://papago.apigw.ntruss.com/nmt/v1/translation"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": CLIENT_ID,
        "X-NCP-APIGW-API-KEY": CLIENT_SECRET,
        "Content-Type": "application/json"
    }
    payload = {"source": source, "target": target, "text": text}

    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        resp.raise_for_status()  # HTTP 에러(4xx/5xx) 자동 예외 발생
        j = resp.json()
        # 안전하게 키 접근
        translated = j.get("message", {}).get("result", {}).get("translatedText")
        if translated:
            return translated
        else:
            # 예상 구조가 아닌 경우 디버그용 출력
            print("❌ 번역 결과 구조가 예상과 다릅니다:", j)
            return text
    except requests.exceptions.RequestException as e:
        print(f"❌ 번역 요청 오류: {e}")
        # 디버그: 가능한 경우 응답 본문도 출력
        try:
            print("응답 텍스트:", resp.text)
        except Exception:
            pass
        return text
    except ValueError as e:
        # JSON 파싱 실패
        print(f"❌ JSON 파싱 오류: {e}")
        return text

In [ ]:
# 동화 설정

# 동화 제목
story_title = input('동화 제목 : ')

# 페이지 수 설정
while True:
    try:
        pages = int(input("몇 페이지 ? (숫자만 입력하세요): "))
        if pages > 10:
            print("❌ 10페이지 이하로 해주세요")
            continue
        elif pages < 1:
            print("❌ 1페이지 이상으로 해주세요")
            continue
        print(f"페이지 수는 {pages}장 입니다")
        break
    except ValueError:
        print("❌ 숫자가 아닙니다. 다시 입력해주세요!")

In [ ]:
# 동화 내용 저장할 리스트 (딕셔너리 형태)
story_pages = []

# 각 페이지별로 내용 입력받기
for i in range(pages):
    print(f"\n📖 {i+1}페이지 내용:")
    print("-" * 30)

    while True:
        content = input(f"{i+1}페이지 >> ").strip()

        if not content:  # 빈 내용 체크
            print("❌ 내용을 입력해주세요!")
            continue

        print("🔄 번역 중..." if USE_PAPAGO else "📝 저장 중...")
        translated_text = papago_translate(content)

        # 딕셔너리 형태로 저장
        page_data = {
            'page': i + 1,
            'text': content,
            'prompt': translated_text
        }

        story_pages.append(page_data)
        print(f"✅ {i+1}페이지 저장 완료!")
        print(f"   원문: {content}")
        print(f"   번역: {translated_text}")
        break

In [ ]:
for page_data in story_pages:
    print(f"\n📖 {page_data['page']}페이지:")
    print(f"   한글: {page_data['text']}")
    print(f"   영어: {page_data['prompt']}")

print(f"\n✨ 총 {len(story_pages)}페이지의 동화가 완성되었습니다!")

In [ ]:
story_pages

In [ ]:
# 각 페이지별로 이미지 생성
for page_data in story_pages:
    page_num = page_data['page']
    prompt = page_data['prompt']
    korean_text = page_data['text']

    print(f"\n🎨 {page_num}페이지 이미지 생성 중...")
    print(f"   프롬프트: {prompt}")

In [ ]:
# 출력 디렉토리 생성
import os
from IPython.display import display, Image as IPImage
import matplotlib.pyplot as plt

output_dir = f"/content/{story_title.replace(' ', '_')}_images"
os.makedirs(output_dir, exist_ok=True)

print(f"📁 이미지 저장 폴더: {output_dir}")
print("=" * 50)

# 각 페이지별로 이미지 생성
generated_images = []  # 생성된 이미지 정보 저장

for page_data in story_pages:
    page_num = page_data['page']
    prompt = page_data['prompt']
    korean_text = page_data['text']

    print(f"\n🎨 {page_num}페이지 이미지 생성 중...")
    print(f"   한글: {korean_text}")
    print(f"   프롬프트: {prompt}")
    print("-" * 30)

    try:
        # 이미지 생성 (SDXL 설정)
        image = pipe(
            prompt=prompt,
            height=1024,
            width=1024,
            num_inference_steps=25,  # SDXL은 보통 25-30 스텝
            guidance_scale=7.5,      # SDXL 권장값
            generator=torch.manual_seed(42 + page_num)  # 페이지별 다른 시드
        ).images[0]

        # 이미지 저장
        filename = f"page_{page_num:02d}.png"
        filepath = os.path.join(output_dir, filename)
        image.save(filepath)

        # 생성된 이미지 정보 저장
        generated_images.append({
            'page': page_num,
            'filepath': filepath,
            'korean_text': korean_text,
            'prompt': prompt,
            'image': image
        })

        print(f"✅ {page_num}페이지 완료! 저장 위치: {filepath}")

        # 메모리 정리 (코랩에서 중요!)
        torch.cuda.empty_cache()

    except Exception as e:
        print(f"❌ {page_num}페이지 생성 실패: {e}")
        # 에러가 발생해도 메모리 정리
        torch.cuda.empty_cache()
        continue

print(f"\n🎉 모든 이미지 생성 완료!")
print(f"📁 저장 폴더: {output_dir}")
print(f"📊 총 {len(generated_images)}개 이미지 생성됨")

# 생성된 이미지들을 바로 확인
print("\n" + "=" * 50)
print("🖼️  생성된 이미지 미리보기")
print("=" * 50)

# matplotlib으로 이미지 표시
fig, axes = plt.subplots(1, len(generated_images), figsize=(5*len(generated_images), 5))
if len(generated_images) == 1:
    axes = [axes]  # 단일 이미지일 때 리스트로 변환

for i, img_data in enumerate(generated_images):
    axes[i].imshow(img_data['image'])
    axes[i].set_title(f"페이지 {img_data['page']}\n{img_data['korean_text'][:20]}...",
                     fontsize=10, pad=10)
    axes[i].axis('off')

plt.tight_layout()
plt.show()